<center><img src="2022-12-04-10-47-03.png" width="1500" height="800"/></center>


# Tackling the Health Crises in Africa

The lack of access to adequate medical resources and facilities have led to the significant number of deaths. Some of these deaths could have been avoided by 
timely access to a medical professional or close proximity of hospitals.

Health systems across Africa are underfunded and understaffed. Less than half of African citizens (52%) – about 615 million people – have access to the health care they need, the quality of health services across the continent is generally poor, and family planning needs of half the continent’s women and girls are unmet.

Africa is where I will focus on this project. Since there are many causes of death, I will look into three of them namely:
1. Terrorism
2. Drug use disorder and
3. Poisoning

My analysis will narrow down to the top 5 countries in Africa where the above cause of death is high. I will then look at the medical doctors that are available in those countries. 

A breakdown of age group death will be given for each of the top countries.



---


### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as po
po.init_notebook_mode(connected=True)

import plotly.io as pio

pio.templates.default = 'plotly_dark'

### Loading Data

In [2]:
nod_by_cause = pd.read_csv('1. annual-number-of-deaths-by-cause.csv')
nod_by_age = pd.read_csv('2. number-of-deaths-by-age-group.csv')
med_doc = pd.read_excel("3. Medical Doctors Per 10000 population.xlsx", header=2)
continent = pd.read_csv("4. ISO 3166_country-and-continent-codes-list-csv.csv")
world_pop = pd.read_csv('5. World Population.csv')
health_exp = pd.read_excel("6. Current health expenditure (% of GDP).xlsx", header=4)

### Filtering for Africa Continent

In [3]:
# Filtering
africa = continent[continent['Continent_Name'] == 'Africa'] 
# Checking
print(africa.Continent_Name.unique())
# Unique country code to filter other data
uniq_3_code = africa['Three_Letter_Country_Code'].unique()
print(uniq_3_code)

['Africa']
['DZA' 'AGO' 'BWA' 'BDI' 'CMR' 'CPV' 'CAF' 'TCD' 'COM' 'MYT' 'COG' 'COD'
 'BEN' 'GNQ' 'ETH' 'ERI' 'DJI' 'GAB' 'GMB' 'GHA' 'GIN' 'CIV' 'KEN' 'LSO'
 'LBR' 'LBY' 'MDG' 'MWI' 'MLI' 'MRT' 'MUS' 'MAR' 'MOZ' 'NAM' 'NER' 'NGA'
 'GNB' 'REU' 'RWA' 'SHN' 'STP' 'SEN' 'SYC' 'SLE' 'SOM' 'ZAF' 'ZWE' 'SSD'
 'ESH' 'SDN' 'SWZ' 'TGO' 'TUN' 'UGA' 'EGY' 'TZA' 'BFA' 'ZMB']


### Filtering Other Data for African Countries

In [4]:
afr_nod_by_cause =  nod_by_cause[nod_by_cause['Code'].isin(uniq_3_code)].reset_index(drop=True)
afr_nod_by_age = nod_by_age[nod_by_age['Code'].isin(uniq_3_code)].reset_index(drop=True)
afr_med_doc = med_doc[med_doc['ParentLocation'] == 'Africa'].reset_index(drop=True)
afr_world_pop =  world_pop[world_pop['Code'].isin(uniq_3_code)].reset_index(drop=True)
afr_health_exp = health_exp[health_exp['Country Code'].isin(uniq_3_code)].reset_index(drop=True)

### Causes of Death Focus
There are more than 10 causes of deaths, I will focus on the following three:
1. Terrorism (deaths)
2. Drug Use Disorders
3. Poisonings

### FUNCTIONS HEADQUARTER

In [5]:
# Function to plot africa map showing cause of death
def map_plot(data, color):
    fig = px.choropleth(data, locations="Code",
                        color=color, width=500,
                        title=f'Death by {color} in Africa',
                        hover_name="Entity", 
                        labels={'Terrorism (deaths)': 'Count', 
                                'Drug use disorders': 'Count',
                                "Poisonings": 'Count'},
                        hover_data=[color],
                        color_continuous_scale='Rainbow', 
    )
    fig.update_layout(geo_scope="africa", 
                    #   margin={"r":0,"t":0,"l":0,"b":0}
                      )
    fig.show()
    
# Function to visualize yearly cause of death
def line_plot(data, title):
    trace = px.line(data,
        x=data.columns[0],
        y=data.columns[1],
        title=title,
        width=900,
        markers=True,
    )
    trace.show()
    
# Function to visualize top 5 countries
def top5_barChart(data, title):
    trace = px.bar(data,
        x=data.columns[0],
        y=data.columns[2],
        text_auto='.3s',
        title=title,
        width=900, labels={'Entity':''}
    )
    trace.update_traces(textposition='outside', cliponaxis=False)
    trace.update_yaxes(showticklabels=False)
    trace.show()
  
# Function to reshape health expenditure data
def reshaper(top_data):
    country_code = top_data['Code'].unique()
    coutry_health_exp = health_exp[health_exp['Country Code'].isin(country_code)]
    coutry_health_exp.set_index('Country Name', inplace=True)
    long_df = pd.melt(coutry_health_exp, id_vars=['Country Code', 'Indicator Name'], value_name='Health Expenditure(% of GDP)', var_name='Year', ignore_index=False)
    long_df.reset_index(drop=False, inplace=True)
    long_df.groupby('Country Name')['Health Expenditure(% of GDP)'].sum()
    reshaped_df = long_df.pivot_table(index='Year', values='Health Expenditure(% of GDP)', columns='Country Name')
    return reshaped_df  
  
# Function to visualize top 5 countries with health expenditures  
def add_year(region, color, data):
    fig.add_trace(go.Scatter(x=data.index, y=data[region],
                             mode="lines+markers",
                             marker= dict(
                                 symbol="diamond", size=3
                             ),
                            name=region, connectgaps=True,
                            line=dict(color=color, width=2)))

# Function to reshape age group data
def reshaper1(top_data):
    country_code = top_data['Code'].unique()
    coutry_health_exp = nod_by_age[nod_by_age['Code'].isin(country_code)]
    long_df = pd.melt(coutry_health_exp, id_vars=['Entity' ,'Code', 'Year'], value_name='Death Count', var_name='Age', ignore_index=False)
    reshaped_df = long_df.groupby(['Entity', 'Age'], as_index=False)['Death Count'].mean()
    return reshaped_df  

# Function to visualize age group death in top 5 country for each cause of death
def group_bar(data, title='Age Group Death'):
    fig = px.bar(
            data,
            x='Entity',
            y='Death Count',
            color='Age', width=1100,
            barmode='group', title=title,
            labels={'Entity':''},
        )
    fig.update_layout(legend=dict(orientation='h'))
    return fig

# Function to merge cause of the column with health expenditure
def subset_merger(column):
    sub = afr_nod_by_cause[['Entity', 'Code', 'Year', column]]
    melt_health = pd.melt(health_exp, id_vars=['Country Name', 'Country Code', 'Indicator Name'], value_name='Health Expenditure(% of GDP)', var_name='Year', ignore_index=False)
    sub_melt_health = melt_health[['Country Code', 'Year', 'Health Expenditure(% of GDP)']]
    merged = pd.merge(sub, sub_melt_health, 'inner', left_on=['Code', 'Year'], right_on=['Country Code', 'Year'])
    return merged

# Function to plot relationship between cause and health expenditure
def cause_health_relationship(data):
    fig = px.scatter(data, y=data.columns[3], x='Health Expenditure(% of GDP)', color='Year',
                     hover_name='Entity')
    fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(
        title=f'Realtionship between {data.columns[3]} and Health Expenditure', width=850,
        xaxis = dict(
            tickvals = [5, 10, 15, 20],
            ticktext = ['5%', '10%', '15%', '20%']
        )
    )
    return fig.show()

### DEATH BY TERRORISM
Terrorism is defined in the [Oxford Dictionary](https://www.lexico.com/en/definition/terrorism) as “the unlawful use of violence and intimidation, especially against civilians, in the pursuit of political aims.” [Read More](https://ourworldindata.org/terrorism)

Below are analysis based on terrorism. We will start with death by terrorism across Africa.

#### Death by Terrorism in Africa
Well, from the scale below we can see that there are more than 7000 death by terrorism, and Nigeria has the highest number of deaths. Statistics below show that **on average, 134 people die as a result of terrorism in Africa** while _7,781_ is the highest number of people to die as a result of this cause (terrorism)

This large death occured in Nigeria in year 2014. Research was done, in year 2014 Nigeria suffered a lot from the militant insurgent group. It was in this year (April 2014) that Boko Haram abducted nearly 300 school girls in the town of Chibook. [Click to Read More](https://www.hrw.org/world-report/2015/country-chapters/nigeria)

In [6]:
map_plot(afr_nod_by_cause, "Terrorism (deaths)")

### Some Statistics on Death by Terrorism

In [7]:
# Terrorism Statistics
afr_nod_by_cause['Terrorism (deaths)'].describe()

count     670.000000
mean      134.388060
std       473.361558
min         0.000000
25%         1.000000
50%        13.000000
75%        90.000000
max      7781.000000
Name: Terrorism (deaths), dtype: float64

In [8]:
# Country with highest number of death by terrorism
afr_nod_by_cause[afr_nod_by_cause['Terrorism (deaths)'] == 7781]

,Entity,Code,Year,Number of executions (Amnesty International),Meningitis,Alzheimer's disease and other dementias,Parkinson's disease,Nutritional deficiencies,Malaria,Drowning,...,Chronic kidney disease,Poisonings,Protein,Terrorism (deaths),Road injuries,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,"Fire, heat, and hot substances",Acute hepatitis
1165,Nigeria,NGA,2014,NaN,52114.0,10623.0,2945.0,7149.0,218689.0,3443.0,...,18290.0,3789.0,6593.0,7781.0,18723.0,23860.0,39342.0,68129.0,4049.0,2935.0


We've seen death by terrorism for each country. 

Below we can see how it has evolve on a yearly basis. There is a high peak in year 2014 and we've seen the country (Nigeria) that made go that high. But from the available data, we can see that there has been no death in year 2018 and 2019. 

In [9]:
yearly_terrorism = nod_by_cause.groupby('Year', as_index=False)['Terrorism (deaths)'].sum()

line_plot(yearly_terrorism, 'Yearly Deaths by Terrorism in Africa')

#### Top 5 Countries with the Highest Number of Death by Terrorism

They include:
1. Nigeria with a total of 22,674 people
2. Algeria with a total of 11,066 people
3. Somalia with a total of 10,242 people
4. Burundi with a total of 4,205 people
5. Democratic Republic of Congo with a total of 4,069

Algeria the second highest is below 50% of Nigeria's count. Further analysis cann be done to see if the death rate has decrease in NIgeria.

In [10]:
terrorism_top5 = afr_nod_by_cause.groupby(['Entity', 'Code'], as_index=False)['Terrorism (deaths)'].sum().sort_values(by='Terrorism (deaths)', ascending=False)[:5]
top5_barChart(terrorism_top5, 'Top 5 African Country with the Highest Number of Death by Terrorism')

Let's take a look at medical doctors in this countries 

In [11]:
terror_country = terrorism_top5['Entity'].unique()
terror_country_code = terrorism_top5['Code'].unique()
med_doc_terror = afr_med_doc[afr_med_doc['Location'].isin(terror_country)]

In [12]:
afr_med_doc['Indicator'].unique()


array(['Medical doctors (per 10,000)', 'Medical doctors (number)',
       'Generalist medical practitioners (number)',
       'Specialist medical practitioners (number)',
       'Medical doctors not further defined (number)'], dtype=object)

In [13]:
med_doc_terror.groupby(['Location'])['Indicator'].nunique()

Location
Algeria    5
Burundi    5
Nigeria    5
Name: Indicator, dtype: int64

From the above, we can see that there are five unique medical indicator in Africa. For the top 5 countries with high death by terrorism, three of them [Nigeria, Algeria and Burundi] has medical doctors while the other two countries [Somalia and Democratic Republic of Congo] has no doctor at all. 

We don't know if the death in Somalia and Democratic Republic of Congo was instant or as a result of injuries sustained from terrorist attacks. If some were as a result of injuries, there is a high probaility of death reduction if there were medical doctors available.

Further investigation can be carried out to be certain that there are really no medical doctors in the countries

Below shows the health expenditures for the 5 countries. Health financing is a critical component of health system. 

We can see that Burundi has spent up to 10% of their GDP on health, twice. On the other end, Nigeria and Congo Republic has not spent up to 6% of their GDP on Health. Algeria falls in between with approx. 7% as the highest percent of GDP spent on Health.

Remember Nigeria had a high number of death in 2014, but we can see here that the percent of GDP spent on health during the year 2014 did not rise, rather it decrease 2.11%. It's obvious that death by terrorism doesn't affect health expenditure. We should probably check the relationship to be certain. 

In [14]:
data = reshaper(terrorism_top5)

fig = go.Figure()
add_year('Algeria', 'firebrick', data)
add_year('Burundi', 'royalblue', data)
add_year('Congo, Dem. Rep.', 'goldenrod', data)
add_year('Nigeria', 'darkturquoise', data)
# add_year(2021, 'forestgreen')
fig.update_layout(title='Health Expenditure in the top 5 countries with highest death by terrorism', width=850,
                  yaxis_title = '% of GDP',
                  yaxis = dict(
                      tickvals = [2, 4, 6, 8, 10],
                      ticktext = ['2%', '4%', '6%', '8%', '10%']
                  )
                  )
fig.show()

Our inference was true, there don't seem to be a linear relationship between death by terrorism and health expenditure. Expenditures is clustered between 1% to 10%. 

There seem to be an outlier in terrorism death, it represent the death in Nigeria which we'vw confirmed basd on the incident. 

The outlier for health expenditure represent Sierra Leone. They spent 16.5% of their GDP on health in Year 2016, surprisingly there was only one death by terrorism for that year. 

With the visual below and the statement above, we can conclude that death by terrorism does not affect how much a country spent on health. 

In [15]:
merged_data = subset_merger('Terrorism (deaths)')

cause_health_relationship(merged_data)

#### Death by Age Group in the Five Countries for Terrorism
We've seen the top 5 countries with high number of death by terrorism. Below shows the number of death by age group for each countries.
 
In Nigeria Children there are 935,818 deaths under age 5 which is the highest amongst five country in all age category. We can also see that there is low number death for 5-14yrs in all countries.

Burundi has the lowest number of death in all age group compared to the other four countries

In [16]:
data = reshaper1(terrorism_top5)

group_bar(data, 'Death by Age Group for Top 5 Country (Terrorism)')

### Recommendation:

* **Each nation in Africa should continue to maintain peace, and also ensure the security agencies are working effectively to eliminate any sign of terrorist organization.**
* **In place like nigeria, there are some locaions that doesn't have police stations, there probability of terrorist attacking those places are high. They government should look into the rural area that has no official governmet agencies.**

Implementing the above would help eliminate any form of attack.

### DEATH BY DRUG USE DISORDER
We've seen alot of insights about death cause by terrorism, and we've explore the top 5 countries in that category.

According to World Health Organization (WHO), substance abuse refers to the harmful or hazardous use of psychoactive substances, including alcohol and illicit drugs. One of the key impacts of illicit drug use on society is the negative health consequences experienced by its members. Drug use also puts a heavy financial burden on individuals, families and society.

The evolution of the complex global illicit drug problem is clearly driven by a range of factors. Sociodemographic trends are influential such as the population’s gender, age and the rate of urbanization. [Read More](https://www.afro.who.int/health-topics/substance-abuse)

Here we would consider the next cause of death on the list, Poisoning. We will see similar analysis like the ones above and will probably divert where necessary for more insights.

Let's start by viewing death by drug use disorder in Africa as a continent

#### Death by Drug Use Disorder in Africa

The map shows Algeria as the country with the highest number of death by drug use, followed by Morocco then South Africa. We will see more about this country later on. 

Have you noticed that there is low number of death as a result of drug use compare to terrorism. Here the maximum number of death is above 500 whereas in terrorism its over 7000.

Below are the few country with low number of death in this category:
1. Guinea Bissau (2)
2. Equatorial Guinea (2)
3. Liberia (3)
4. Gabon (4)
5. Sierra Leone (4)

Let's see some statistics for this cause of death.


In [17]:
map_plot(afr_nod_by_cause, 'Drug use disorders')

### Some Statistics on Death by Drug Use
On average, 46 people die as a result of Drug Use in Africa with a median of 12.

Maximum and minimum number of death are 526 and 0 respectively.

In [18]:
# Drug Use Statistics
afr_nod_by_cause['Drug use disorders'].describe()

count    1620.000000
mean       46.481481
std        79.924626
min         0.000000
25%         5.000000
50%        12.000000
75%        44.000000
max       526.000000
Name: Drug use disorders, dtype: float64

From the below line plot, we can see that number of death by drugs use has been increasing for the past decade. Compared to the previous cause, there are larger number of deaths in this category. The last data point reveals that 833,990 people have died as a result of drug use in 2019. 

Which country is contributing to this? When will the number start decreasing? Further analysis can be done to answer this questions and others.

The second question is more of machine learning which is outside this scope. Let's look at the top 5 countries in this cause of death category (Drug Use)

In [19]:
yearly_drugUse = nod_by_cause.groupby('Year', as_index=False)['Drug use disorders'].sum()
line_plot(yearly_drugUse, 'Yearly Deaths by Drug Use Disorder in Africa')

#### Top 5 countries with the Highest Number of Death by Drug Use Disorder

They include:
1. Algeria with a total of 10,612 people
2. Morocco with a total of 7,718 people
3. South Africa with a total of 7,414 people
4. Sudan with a total of 6,526 people
5. Nigeria with a total of 4,897

From the plot below, we can see that Algeria has the highest number of death in the category, and Nigeria is among the top 5 country. South Africa is 4% lower than Morocco.

Let's look at the health expenditure for this set of countries


In [20]:
drug_use_top5 = afr_nod_by_cause.groupby(['Entity', 'Code'], as_index=False)['Drug use disorders'].sum().sort_values(by='Drug use disorders', ascending=False)[:5]
top5_barChart(drug_use_top5, 'Top 5 African Country with the Highest Number of Death by Drug Use Disorder')

Algeria and Nigeria were in the previous cause of death analysis, and both country are in this category too. 

We can see that in this cause of death categogry (Drug Use), South Africa are spending more of their GDP in health compared to other countries. South Africa has never spent below 7% of their GDP on health.

We've established earlier that there is no linear relationship between daeath by terrorism and health expenditure. Is this true for death by Drug Use? Let's find out

In [21]:
data = reshaper(drug_use_top5)

fig = go.Figure()
add_year('Algeria', 'firebrick', data)
add_year('Morocco', 'royalblue', data)
add_year('Nigeria', 'goldenrod', data)
add_year('South Africa', 'darkturquoise', data)
add_year('Sudan', 'forestgreen', data)
fig.update_layout(title='Health Expenditure in the top 5 countries with highest death by Drug Use Disorder', width=850,
                  yaxis_title = '% of GDP',
                  yaxis = dict(
                      tickvals = [2, 3, 4, 5, 6, 7, 8, 9,],
                      ticktext = ['2%', '3%', '4%', '5%', '6%', '7%', '8%', '9%']
                  )
                  )
fig.show()

Our above question has been answered with the visual below. There is no linear relationship between drug use an expenditure, maybe this is applicable to all cause of deaths. Further anlysis can be carried to confirm this theory.

The scatter plot also show that some country spent more than 16.5% of their GDP on health. Sierra Leone is the country in this case. Hover to see more details. 

In [22]:
merged_data = subset_merger('Drug use disorders')

cause_health_relationship(merged_data)

#### Death by Age Group in the Five Countries for Drug Use

Nigeria has the highest number of death in all age group compared to other country.

Algeria has lowest number of death in all age group compared to other country

In [23]:
data = reshaper1(drug_use_top5)

group_bar(data, 'Death by Age Group for Top 5 Country (Drug Use Disorder)')

### Recommendation
* **For this cause,  the five country can carry out investigation to know what type of drugs are kiling people in their nation.**
* **Organization in charge of overseeing companies that produces drugs should implement some measures that will ensure drug are not illegally sold**
* **Currently, death by this cause is increasing in Africa as a whole. Investigatio should be carried out to know which drug is killing Africans and implement measures to elimninate as a continent.**

### DEATH BY POISONING

"Poisoning is defined as damaging physiologic effects from substances classified as illicit drugs, pharmaceutical drugs (both prescribed and over the counter), chemicals, such as pesticides, heavy metals (such as mercury), gases/vapors, such as carbon monoxide, or household substances, such as antifreeze or bleach. This classification is somewhat unsatisfactory in that many pharmaceutical drugs, such as oxycodone, are often sold illicitly, or otherwise obtained by persons without a valid prescription. Among adults, chemicals, household substances, and gases and vapors are almost exclusively used in suicides, and are rarely the cause of an unintentional death". [Read More](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2563476/)

Lastly, let's look into death cause by poisoning.

As usual we would see number of death across Africa then dig deeper

#### Death by Poisoning in Africa

From the map we can see that Nigeria is the country with the highest number of death in this category, followed by Ethiopia.

From the color scale, we can see that the maximum number is above that of death by drug, but it's not greater than than that of death by terrorism.

Let's see some statistics

In [24]:
map_plot(afr_nod_by_cause, "Poisonings")

### Some Statistics on Death by Poisoning
On average, 378 people die as a result of Poisoning in Africa with a median of 210.

Maximum and minimum number of death are 4361 and 0 respectively.

Let's see the trend for this cause of death

In [25]:
# Poisoning Statistics
afr_nod_by_cause['Poisonings'].describe()

count    1620.000000
mean      378.929630
std       668.755446
min         0.000000
25%        38.000000
50%       210.000000
75%       403.250000
max      4361.000000
Name: Poisonings, dtype: float64

From the line plot, we can see that the number of death by poison has been decreasing since year 2008 after a little spike (0.33% increase from 2007). 

The highest number of death occured in 2015, a total of 519,820 people died because of poison. Further analysis can be carried out for more insights. 

Based on our current data, the lowest number of death (434,114) occured in 2019, 

In [26]:
yearly_poisonings = nod_by_cause.groupby('Year', as_index=False)['Poisonings'].sum()
line_plot(yearly_poisonings, 'Yearly Deaths by Poisoning in Africa')

#### Top 5 countries with Highest Number of Death by Poisoning

They include:
1. Nigeria with a total of 107,604 people.
2. Ethiopia with a total of 101,065 people
3. Democratic Republic of Congo with a toal of 37,676 people
4. Tanzania with a total of 27,430 people
5. Sudan with a total of 25,388 people

It looks like Nigeria and Ethiopia are competing, literally. Ethiopia is 6% lower than Nigeria.

Congo Republic, Tanzania and Sudan are below 50% of Ethiopia. Tanzana is higher than Sudan with 7.4%.

In [27]:
poisoning_top5 = afr_nod_by_cause.groupby(['Entity', 'Code'], as_index=False)['Poisonings'].sum().sort_values(by='Poisonings', ascending=False)[:5]
top5_barChart(poisoning_top5, 'Top 5 African Country with the Highest Number of Death by Poisoning')

From the line plot, Tanzania spent approximately 7.5% of their GDP on health in year 2006. Starting from that year, their expense on health has reduce.

In year 2019 Nigeria, Tanzania, Congo Republic and Ethiopia, spent below 4% of their GDP on health.

Let's look at the relationship 

In [28]:
data = reshaper(poisoning_top5)

fig = go.Figure()
add_year('Congo, Dem. Rep.', 'firebrick', data)
add_year('Ethiopia', 'royalblue', data)
add_year('Nigeria', 'goldenrod', data)
add_year('Sudan', 'darkturquoise', data)
add_year('Tanzania', 'forestgreen', data)
fig.update_layout(title='Health Expenditure in the top 5 countries with highest death by Poisoning', width=850,
                  yaxis_title = '% of GDP',
                  yaxis = dict(
                      tickvals = [2, 3, 4, 5, 6, 7, 8, 9],
                      ticktext = ['2%', '3%', '4%', '5%', '6%', '7%', '8%', '9%']
                  )
                  )
fig.show()

There is a cluster between range 2,500 - 4,100 of death count. it's worth investigating.

In [29]:
merged_data = subset_merger('Poisonings')

cause_health_relationship(merged_data)

#### Death by Age Group in the Five Countries for Poisoning
From the chart below, Sudan has the lowest number of death ineach age group compared to other country.

Nigeria still dominate here.

In [30]:
data = reshaper1(poisoning_top5)

group_bar(data, 'Death by Age Group for Top 5 Country (Poisoning)')

### Recommendation:

* **Its a good thing that the number of death by poisoning is reducing across Africa. Any form of poisons, chemical, pesticides, heavy metals, etc, should be contained before exposure.**
* **Some death could be as a result of poisonous water in rural village. Government can carry the project of ensuring its citizen in the remote area have access to good water. They can work towards build water facility in those area.**
* **Chemical companies should make sure their environment is not exposed to what they are producing.**

## SUMMARY

![](2022-12-04-14-53-13.png)

1. In Africa, average of _134_ people die as a result of _terrorism_,  average of _46_ people die as a result of _drug use disorder_ while average of _379_ people die as a result of _poisoming_.

2. Below are the top 5 country for each cause of death analyzed above:

|     | Terrorism | Drug Use Disorder | Poisoning |
| --- | --- | --- | --- |
| 1 | Nigeria [22,674] | Algeria [10,612] | Nigeria [107,604] |
| 2 | Algeria [11,066] | Morocco [7,718] | Ethiopia [101,065] |
| 3 | Somalia [102,42] | South AFrica [7,414] | Congo Republic [37,676] |
| 4 | Burundi [4,205] | Sudan [6,526] | Tanzania [27,430] |
| 5 | Congo Republic [4,069] | Nigeria [4,897] | Suda [25,388] |

3. Based on the given data, there has been no death by terrorism in year 2018 and 2019.
4. In year 2015 Sierra Leone spent 20.4% of their GDP on health services
5. In Nigeria, there is a huge number of death with children under age 5

LIMITATION:
* The type of drugs that cause death is not available. We could have narrow it down to who's producing such drugs and in what location is it killing the most.
* Types os poison is not available. We could narrow it down to know if it's from companies or individuals.
* Data about each country GDP is was not used. This would have narrow down why some health expenditure are decreasing and increasing. 

